# Data Understanding

In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re

In [5]:
#data loader class
class DataLoader:
    def __init__(self,dir_name,file_name):
        self.dir_name=dir_name
        self.file_name = file_name
    
 
    def read_csv(self):
        os.chdir(self.dir_name)
        tweets_df=pd.read_csv(self.file_name)
        return tweets_df
  
    

In [6]:
#object creation
DataLoader_obj= DataLoader('../data/','processed_tweet_data.csv')

# Columns of the data

In [7]:
tweets_df=DataLoader_obj.read_csv()
tweets_df.dropna()

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
5,Sun Aug 07 22:31:02 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @benedictrogers: We must not let this happe...,0.200000,0.500000,en,41770,36,GraceCh15554845,207,54,False,['Taiwan'],['benedictrogers'],"Melbourne, Victoria"
11,Sun Aug 07 22:30:35 +0000 2022,"<a href=""https://mobile.twitter.com"" rel=""nofo...","Wilson Chinonso Blog: Nigerian tribes, the lis...",0.000000,0.000000,en,134,0,wilson_chnns,28,265,False,"['China', 'ChinaTaiwan', 'ManUnited']",[],Imo State Nigeria
19,Sun Aug 07 22:30:01 +0000 2022,"<a href=""https://buffer.com"" rel=""nofollow"">Bu...",27.89US $ 17% OFF|Usb Condenser Microphone For...,0.000000,0.000000,en,265,0,doos94619918,1936,4792,False,"['aliexpress', 'USA', 'uk', 'RT', 'Europe', 'U...",[],"United States New York,"
36,Sun Aug 07 22:26:25 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @ChinaInfo777: #PinkFloyd Roger Waters tell...,0.000000,0.000000,en,91839,5,nhohn2011,870,508,False,"['PinkFloyd', 'Taiwan', 'China']",['ChinaInfo777'],"Florida, USA"
39,Sun Aug 07 22:25:37 +0000 2022,"<a href=""http://twitter.com/download/android"" ...","RT @WilliamYang120: ""For too long, #Taiwan has...",-0.025000,0.200000,en,46145,84,hoggothoaryhost,44,60,False,['Taiwan'],['WilliamYang120'],Hong Kong
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21981,Sat Aug 06 18:04:09 +0000 2022,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @jenniferatntd: Head of #Taiwan's #missile ...,-0.200000,0.400000,en,6660,99,threadmaxwhispe,657,864,False,"['Taiwan', 'missile']",['jenniferatntd'],Land of Ethan South Dakota
21989,Sat Aug 06 18:03:48 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",Minister Wu is crystal clear in his @BBCNews i...,0.158333,0.419444,en,3129,0,TECO_Toronto,955,202,False,"['Taiwan', 'StandWithTaiwan', 'DefendDemocracy']","['BBCNews', 'SpeakerPelosi']","Toronto, Canada"
21990,Sat Aug 06 18:03:47 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @SpokespersonCHN: #PLA Live-fire military d...,-0.100000,0.250000,en,175203,405,mumaralid,1164,605,True,"['PLA', 'Taiwan']",['SpokespersonCHN'],Driver
21992,Sat Aug 06 18:03:33 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...",RT @jenniferatntd: Head of #Taiwan's #missile ...,-0.200000,0.400000,en,14305,99,9thousandbytes,401,858,False,"['Taiwan', 'missile']",['jenniferatntd'],Northern Virginia


# Number of rows

In [8]:
len(tweets_df)

22000

# The first 5 rows

In [9]:
tweets_df.head()

,created_at,source,original_text,polarity,subjectivity,lang,favorite_count,retweet_count,original_author,followers_count,friends_count,possibly_sensitive,hashtags,user_mentions,place
0,Sun Aug 07 22:31:20 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @i_ameztoy: Extra random image (I):\n\nLets...,-1.250000e-01,0.190625,en,15760,2,i_ameztoy,20497,2621,NaN,['City'],['i_ameztoy'],NaN
1,Sun Aug 07 22:31:16 +0000 2022,"<a href=""http://twitter.com/download/android"" ...",RT @IndoPac_Info: #China's media explains the ...,-1.000000e-01,0.100000,en,6967,201,ZIisq,65,272,NaN,"['China', 'Taiwan']",['IndoPac_Info'],NaN
2,Sun Aug 07 22:31:07 +0000 2022,"<a href=""http://twitter.com/download/android"" ...","China even cut off communication, they don't a...",0.000000e+00,0.000000,en,2166,0,Fin21Free,85,392,NaN,['XiJinping'],['ZelenskyyUa'],Netherlands
3,Sun Aug 07 22:31:06 +0000 2022,"<a href=""http://twitter.com/download/android"" ...","Putin to #XiJinping : I told you my friend, Ta...",1.000000e-01,0.350000,en,2166,0,Fin21Free,85,392,NaN,['XiJinping'],[],Netherlands
4,Sun Aug 07 22:31:04 +0000 2022,"<a href=""http://twitter.com/download/iphone"" r...","RT @ChinaUncensored: I’m sorry, I thought Taiw...",-6.938894e-18,0.556250,en,17247,381,VizziniDolores,910,2608,NaN,[],['ChinaUncensored'],"Ayent, Schweiz"


# Check for missing values

In [10]:
print("The number of missing value(s) based on columns:\n{}".format(tweets_df.isnull().sum()))
print("The sum of missing value(s) is:\n{}".format(tweets_df.isnull().sum().sum()))

The number of missing value(s) based on columns:
created_at                0
source                    0
original_text             0
polarity                  0
subjectivity              0
lang                      0
favorite_count            0
retweet_count             0
original_author           0
followers_count           0
friends_count             0
possibly_sensitive    15809
hashtags                  0
user_mentions             0
place                  9893
dtype: int64
The sum of missing value(s) is:
25702


# Data Cleaning

In [11]:
import sys
sys.path.append('../')


In [12]:
from clean_tweets_dataframe import Clean_Tweets

In [13]:
# Fill missing values 
processed_tweets = tweets_df.fillna("")

In [14]:
import re

def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweet: cleaned tweet

    """
    # remove hashtags
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)
    # remove @ handles
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks    
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)

    return tweet

In [20]:
def data_cleaner(df: pd.DataFrame, save=False) -> pd.DataFrame:
    """Data Cleaner Function.
    Input:
        df: Pandas Dataframe
        save: Boolean value
    Output:
        df: Cleaned Dataframe

    """
    Tweet_cleaner = Clean_Tweets(df)
    df = Tweet_cleaner.remove_non_english_tweets(df)
    df = Tweet_cleaner.drop_duplicate(df)
    df = Tweet_cleaner.drop_unwanted_column(df)
    df = Tweet_cleaner.drop_unwanted_column(df)
    df = Tweet_cleaner.convert_to_datetime(df)
    df = Tweet_cleaner.convert_to_numbers(df)
    df['clean_text'] = df['original_text'].apply(process_tweet)
    df['clean_text'] =  df['clean_text'].astype(str)
    df['clean_text'] = df['clean_text'].apply(lambda x: x.lower())
    df['clean_text']= df['clean_text'].apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation)))


    if save:
        try: 
            df.to_csv('../data/cleaned_tweet_data.csv', index=False)
            
            print('File Successfully Saved.!!!')
        
        except Exception as e:
            print("Save failed...",e)
    return df

In [21]:
cleaned_df = data_cleaner(tweets_df, save=True)

Automation in Action...!!!
File Successfully Saved.!!!


In [22]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21997 entries, 0 to 21999
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   created_at          21997 non-null  datetime64[ns, UTC]
 1   source              21997 non-null  object             
 2   original_text       21997 non-null  object             
 3   polarity            21997 non-null  float64            
 4   subjectivity        21997 non-null  float64            
 5   lang                21997 non-null  object             
 6   favorite_count      21997 non-null  int64              
 7   retweet_count       21997 non-null  int64              
 8   original_author     21997 non-null  object             
 9   followers_count     21997 non-null  int64              
 10  friends_count       21997 non-null  int64              
 11  possibly_sensitive  6190 non-null   object             
 12  hashtags            21997 non-nu

In [29]:
sentence_list = [tweet for tweet in cleaned_df['clean_text'].head()]
word_list = [sent.split() for sent in sentence_list]

word_to_id = corpora.Dictionary(word_list) #generate unique tokens
    
corpus= [word_to_id.doc2bow(tweet) for tweet in word_list]

In [31]:
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)], [(7, 2), (10, 1), (15, 5), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1)], [(15, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1)], [(5, 2), (15, 1), (35, 1), (52, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 2)], [(5, 1), (27, 1), (60, 1), (70, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1)]]


In [33]:
id_words = [[(word_to_id[id], count) for id, count in line] for line in corpus]

In [34]:
print(id_words)

[[('coast', 1), ('district', 1), ('extra', 1), ('focus', 1), ('gt', 1), ('i', 1), ('image', 1), ('in', 1), ('lets', 1), ('longjing', 1), ('of', 1), ('one', 1), ('random', 1), ('specific', 1), ('taichung', 1), ('the', 1), ('very', 1), ('western', 1), ('zone', 1), ('…', 1)], [('in', 2), ('of', 1), ('the', 5), ('area', 1), ('drills', 1), ('each', 1), ('explains', 1), ('for', 1), ('labels', 1), ('media', 1), ('military', 1), ('pi…', 1), ('read', 1), ('reasons', 1), ('s', 1), ('strait', 1)], [('the', 2), ('anwer', 1), ('ask', 1), ('but', 1), ('change', 1), ('china', 1), ('clown', 1), ('communication', 1), ('cut', 1), ('dont', 1), ('enters', 1), ('even', 1), ('from', 1), ('here', 1), ('mind', 1), ('off', 1), ('phonecalls', 1), ('putins', 1), ('stage', 1), ('they', 1), ('to', 2), ('us', 1)], [('i', 2), ('the', 1), ('but', 1), ('to', 2), ('a', 1), ('be', 1), ('chinas', 1), ('eyes', 1), ('friend', 1), ('including', 1), ('it', 1), ('like', 1), ('model', 1), ('much', 1), ('my', 1), ('nukes', 1), 

# Topic Modeling using Latent Dirichlet Allocation

In [37]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word=word_to_id ,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [38]:
pprint(lda_model.print_topics())

[(0,
  '0.032*"own" + 0.032*"country" + 0.032*"i" + 0.032*"was" + 0.032*"had" + '
  '0.032*"military" + 0.032*"it" + 0.032*"government" + 0.032*"independent" + '
  '0.032*"because"'),
 (1,
  '0.088*"the" + 0.031*"to" + 0.031*"in" + 0.031*"you" + 0.031*"i" + '
  '0.017*"but" + 0.017*"of" + 0.017*"military" + 0.017*"strait" + '
  '0.017*"area"'),
 (2,
  '0.011*"of" + 0.011*"specific" + 0.011*"extra" + 0.011*"gt" + 0.011*"i" + '
  '0.011*"very" + 0.011*"one" + 0.011*"in" + 0.011*"lets" + 0.011*"the"'),
 (3,
  '0.031*"district" + 0.031*"western" + 0.031*"coast" + 0.031*"taichung" + '
  '0.031*"zone" + 0.031*"…" + 0.031*"image" + 0.031*"longjing" + 0.031*"very" '
  '+ 0.031*"one"'),
 (4,
  '0.052*"to" + 0.051*"the" + 0.028*"even" + 0.028*"clown" + 0.028*"but" + '
  '0.028*"dont" + 0.028*"stage" + 0.028*"china" + 0.028*"ask" + 0.028*"anwer"')]


In [39]:
pprint(lda_model.show_topics(formatted=False))

[(0,
  [('own', 0.03184627),
   ('country', 0.031842645),
   ('i', 0.03183027),
   ('was', 0.031825766),
   ('had', 0.031823944),
   ('military', 0.03182359),
   ('it', 0.03181817),
   ('government', 0.031813342),
   ('independent', 0.03181118),
   ('because', 0.031810854)]),
 (1,
  [('the', 0.08784842),
   ('to', 0.031160245),
   ('in', 0.03115323),
   ('you', 0.031130059),
   ('i', 0.031121919),
   ('but', 0.017011398),
   ('of', 0.016992591),
   ('military', 0.016992526),
   ('strait', 0.016992446),
   ('area', 0.016992433)]),
 (2,
  [('of', 0.010845418),
   ('specific', 0.0108450465),
   ('extra', 0.0108443005),
   ('gt', 0.0108314),
   ('i', 0.010830623),
   ('very', 0.010830491),
   ('one', 0.010829565),
   ('in', 0.010828993),
   ('lets', 0.0108260065),
   ('the', 0.010825024)]),
 (3,
  [('district', 0.031089652),
   ('western', 0.031071771),
   ('coast', 0.031071125),
   ('taichung', 0.031069558),
   ('zone', 0.03106355),
   ('…', 0.031062469),
   ('image', 0.03106179),
   ('lo

# Model Analysis

In [41]:
# Compute Perplexity

#It's a measure of how good the model is. The lower the better. Perplexity is a negative value
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
doc_lda = lda_model[corpus]


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=word_to_id, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n Ldamodel Coherence Score/Accuracy on Tweets: ', coherence_lda)


Perplexity:  -5.003623516922412

 Ldamodel Coherence Score/Accuracy on Tweets:  0.8676158450606035


In [42]:
!pip install pyLDAvis 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 419.8 kB/s eta 0:00:00m eta 0:00:010:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.3/381.3 kB 321.5 kB/s eta 0:00:001m321.5 kB/s eta 0:00:01
  Using cached future-0.18.2.tar.gz (829 kB)
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 349.3 kB/s eta 0:00:00m eta 0:00:010:00:03
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136882 sha256=e28c818e9ff8b11ac3c34f03e2d01325b6b4a210e67d51a00a15155573741b51
  Stored in directory: /home/hmicheal/.cache/pip/wheels/e9/95/74/4766157910829d2fbc6b913850a62cfd15da77a199302cdf9c
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=11a60c4d79175bce6c9e7c27f07